In [1]:
import random
import numpy as np
#import pandas as pd MARCHE PAS
import matplotlib.pyplot as plt

# Structure de données, affichage

In [2]:
# Représentation d'un pseudo-labyrinthe avec deux matrices de murs horizontaux et verticaux
class pseudoLabyrinthe:
    def __init__(self, n, m): # constructeur de la classe pseudolabyrinthe
        self.n = n
        self.m = m
        # Matrice de murs (True = mur, False = passage)
        self.murs_horizontaux = [[False for i in range(m)] for j in range(n-1)]
        self.murs_verticaux = [[False for i in range(n)] for j in range(m-1)]

    # Génère un pseudo-labyrinthe aléatoire
    def genereraleatoire(self):
        for i in range(self.n - 1):
            for j in range(self.m):
                self.murs_horizontaux[i][j] = random.choice([True, False])
        for i in range(self.m - 1):
            for j in range(self.n):
                self.murs_verticaux[i][j] = random.choice([True, False])
    def enceinte(self):
        """
        affiche une cellule de taille M*N, avec la grille
        """
        k = line2d([(0,0), (self.m, 0), (self.m, self.n), (0, self.n), (0,0)], color="red", thickness=2, gridlines=True)
        return k
    def afficher(self):
        graphHorizontal = line2d([(0, 0), (0, 0)])
        graphVertical = line2d([(0, 0), (0, 0)])
        for i in range(len(self.murs_horizontaux)):
            for j in range(len(self.murs_horizontaux[i])):
                if (self.murs_horizontaux[i][j]):
                    graphHorizontal += line2d([(j, i+1), (j+1, i+1)], color="red", thickness=2) # j'ai échangé j et i
        for i in range(len(self.murs_verticaux)):
            for j in range(len(self.murs_verticaux[i])):
                if (self.murs_verticaux[i][j]):
                    graphVertical += line2d([(i+1, j), (i+1, j+1)], color="red", thickness=2)
        return graphHorizontal + graphVertical + self.enceinte()
    def nbmurscorr(self):
        l = np.random.permutation(np.array([0 for i in range(self.n*self.m-1)] 
                                           + [1 for i in range((2*self.n*self.m - self.n - self.m )-(self.n*self.m-1))]))
        t = np.split(l, (self.n*(self.m-1),))
        self.murs_horizontaux = list(np.split(t[1], self.n-1))
        self.murs_verticaux = list(np.split(t[0], self.m-1))
    def estconnexe(self):
        """
         La complexité de l’algorithme de recherhce pour la fonction estconnexe() est O(n * m), où n est le nombre de lignes et m le nombre de colonnes dans le labyrinthe. 
         Cela signifie que dans le pire des cas, l’algorithme visitera chaque cellule une fois.

         On explore chaque nœud (cellule) une fois et chaque arête (passage entre les cellules) une fois.
         Dans un labyrinthe, le nombre total de passages possibles est proportionnel au nombre de cellules, 
        donc l’algorithme doit vérifier chaque cellule et chaque passage adjacent pour s’assurer qu’il a exploré toutes les parties accessibles du labyrinthe.
         Même si certaines cellules sont visitées plus d’une fois à cause des retours en arrière, 
        chaque cellule est traitée une seule fois lorsqu’elle est visitée pour la première fois.
        """
        passage = [[False for i in range(self.m)] for j in range(self.n)]
        def recherche(i, j):
            """
            Algorithme de recherche inspiré d'un DFS
            """
            # Condition d'arrêt de notre boucle récursive
            if i < 0 or i >= self.n or j < 0 or j >= self.m or passage[i][j]:
                return
            passage[i][j] = True  # Marquer la cellule comme visitée
            # Parcourir les cellules adjacentes si il n'y a pas de mur
            if i > 0 and not self.murs_horizontaux[i-1][j]:
                recherche(i-1, j)
            if i < self.n - 1 and not self.murs_horizontaux[i][j]:
                recherche(i+1, j)
            if j > 0 and not self.murs_verticaux[j-1][i]:
                recherche(i, j-1)
            if j < self.m - 1 and not self.murs_verticaux[j][i]:
                recherche(i, j+1)

        recherche(0, 0)  # Commencer le parcours à partir de la cellule (0, 0)
        # Vérifier si toutes les cellules ont été visitées
        return all(all(row) for row in passage)
    def nmurs(self):
        return np.sum(np.array(self.murs_horizontaux), dtype=int) + np.sum(np.array(self.murs_verticaux), dtype=int)
    def estLabyrinthe(self):
        return self.estconnexe() and self.nmurs() == self.n*self.m - self.n - self.m + 1

In [19]:
class labyrinthe(pseudoLabyrinthe):
    def __init__(self, n, m):
        super().__init__(n, m)
    def ajouteraleatoire(self):
        nalea = random.randint(1, self.n-1) 
        malea = random.randint(1, self.m-1)  
        horizontal = random.choice([True, False])
        if horizontal:
            self.murs_horizontaux[nalea - 1][malea] = True
        else:
            self.murs_verticaux[malea - 1][nalea] = True
        return nalea, malea, horizontal # on preserve l'information pour la fonction genereraleatoire
    def genereraleatoirenaif(self):
        """ 
        on adopte une approche itérative avec un algorithme dont la complexité
        est O(inf) au pire des cas, c'est une generation de labyrinthe naive
        >>> deprecated
        """
        self.nbmurscorr()
        while not self.estconnexe():
            self.nbmurscorr()
    # tuples (0,0) sur matrice
    def genereraleatoirebis(self):
        self.murs_horizontaux = [[False for i in range(self.m)] for j in range(self.n-1)]
        self.murs_verticaux = [[False for i in range(self.n)] for j in range(self.m-1)]
        while self.nmurs() != self.n*self.m - self.n - self.m + 1:
            while self.estconnexe():
                info = self.ajouteraleatoire()
            # créer un dico ou on met tous les murs deja posés
            if info[2]:
                self.murs_horizontaux[info[0]-2][info[1]-1] = False
            else:
                self.murs_verticaux[info[1]-2][info[0]-1] = False
    def genereraleatoireter(self):
        self.murs_horizontaux = [[False for i in range(self.m)] for j in range(self.n-1)]
        self.murs_verticaux = [[False for i in range(self.n)] for j in range(self.m-1)]
        logmurs = []
        max_attempts = 100  # Set a maximum number of attempts

        while self.nmurs() != self.n*self.m - self.n - self.m + 1:
            attempts = 0
            while self.estconnexe() and attempts < max_attempts:
                info = self.ajouteraleatoire()
                if info not in logmurs:
                    logmurs.append(info)
                    attempts = 0 #piste à explorer
                else:
                    attempts += 1
            if attempts >= max_attempts:
                break

            if info[2]:
                self.murs_horizontaux[info[0]-2][info[1]-1] = False
            else:
                self.murs_verticaux[info[1]-2][info[0]-1] = False

# test d'affichage et de création d'un labyrinthe

In [18]:
a.murs_verticaux

[[False, False, False, False, False],
 [False, False, False, True, False],
 [False, True, False, False, False],
 [True, False, False, False, False],
 [False, True, False, False, False]]

In [21]:
a = labyrinthe(5, 6)
a.genereraleatoireter()
a.afficher()

KeyboardInterrupt: 

In [ ]:
for i in range(100):
    a.genereraleatoireter()
    if a.estLabyrinthe():
        a.afficher()
    else:
        print(f"Nombre de murs du labyrinthe courant : {a.nmurs()} \nNombre de murs attendu : {a.n*a.m-a.n-a.m+1} \nConnexe = {a.estconnexe()}")
        raise RuntimeError

# Calcul combinatoire sur les labyrinthes

## Vérification

Nous allons vérifier si la génération de notre labyrinthe est uniforme, voici des étapes à suivre : 
* générer un labyrinthe aléatoirement.
* lui assigner un nombre tiré des valeurs de ses matrices
* mettre ce labyrinthe dans un dictionnaire/un tableau pandas
* vérifier la distribution                            
Effectuons ce test sur un labyrinthe 2x3

### moyenne des baricentres de nos matrices

In [158]:
np.array(lab32.murs_horizontaux, dtype = int)

array([[0, 0],
       [0, 0]])

In [83]:
np.mean(np.argwhere(a.murs_horizontaux), axis=None) + np.mean(np.argwhere(a.murs_verticaux), axis=None)/2

36.464702829775796

In [8]:
lab32 = labyrinthe(2,3)

In [6]:
lab32.genereraleatoireter()

NameError: name 'lab32' is not defined

In [148]:
def signature(lab):
    return ((np.mean(np.sum(np.argwhere(np.array(lab.murs_horizontaux, dtype=int)), axis=None))) + (
        np.mean(np.sum(np.argwhere(np.array(lab32.murs_verticaux, dtype=int)), axis=None)))) / 2

In [165]:
for i in range(100):
    lab32.genereraleatoireter()

KeyboardInterrupt: 

In [150]:
dictionnaire = {}
for i in range(100):
    lab32.genereraleatoireter()
    if signature(lab32) not in dictionnaire:
        dictionnaire[signature(lab32)] = 1
    else:
        dictionnaire[signature(lab32)] += 1

KeyboardInterrupt: 

générer les labyrinthes aléatoirement il faut générer des petits labyrinthes aléatoirement puis ajouter des cases à ce labyrinthe et on rajoute dans les nouvelles lignes tout ce qui fait que c'est un labyrinthe !